<a href="https://colab.research.google.com/github/mcgmed/SQL/blob/main/SQLite_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3
import pandas as pd
con = sqlite3.connect('/content/sample_data/test.db')
cur = con.cursor()

## DATA TYPES

*   NULL values mean missing information or unknown.
*   Integer values are whole numbers (either positive or negative). An integer can have variable sizes such as 1, 2,3, 4, or 8 bytes.
*   Real values are real numbers with decimal values that use 8-byte floats.
*   TEXT is used to store character data. The maximum length of TEXT is unlimited.SQLite supports various character encodings.
*   BLOB stands for a binary large object that can store any kind of data. The maximum size of BLOB is, theoretically, unlimited.

*   If a literal has no enclosing quotes and decimal point or exponent, SQLite assigns the INTEGER storage class.
*   If a literal is enclosed by single or double quotes, SQLite assigns the TEXT storage class.
*   If a literal does not have quote nor decimal point nor exponent, SQLite assigns REAL storage class.
*   If a literal is NULL without quotes, it assigned NULL storage class.
*   If a literal has the X’ABCD’ or x ‘abcd’, SQLite assigned BLOB storage class.

In [ ]:
res = cur.execute("SELECT	typeof(100), typeof(10.0), typeof('100'), typeof(x'1000'), typeof(NULL)")
for row in res:
  print(row)

('integer', 'real', 'text', 'blob', 'null')


In [ ]:
cur.execute("""CREATE TABLE test_datatypes (id INTEGER PRIMARY KEY,
                                            val)""")
cur.execute("""INSERT INTO test_datatypes (val)
               VALUES	(1), (2), (10.1),	(20.5),	('A'), ('B'),	(NULL),	(x'0010'), (x'0011')
               """)
con.commit()

In [ ]:
query = "SELECT	id,	val, typeof(val) FROM	test_datatypes"
data = pd.read_sql(query, con)
data

,id,val,typeof(val)
0,1,1,integer
1,2,2,integer
2,3,10.1,real
3,4,20.5,real
4,5,A,text
5,6,B,text
6,7,None,null
7,8,b'\x00\x10',blob
8,9,b'\x00\x11',blob


## DATE & TIME

In [ ]:
cur.execute("CREATE TABLE datetime_text(d1 text, d2 text)")
con.commit()

To get the current UTC date and time value, you pass the now literal string to the function as follows:

In [ ]:
cur.execute("SELECT datetime('now')")
cur.fetchall()

[('2023-01-04 11:45:36',)]

To get the local time, you pass an additional argument  localtime.

In [ ]:
cur.execute("SELECT datetime('now', 'localtime')")
cur.fetchall()

[('2023-01-04 11:45:36',)]

In [ ]:
cur.execute("""INSERT INTO datetime_text (d1, d2)
               VALUES(datetime('now'),datetime('now', 'localtime'))
               """)

In [ ]:
query = "SELECT	* FROM datetime_text"
data = pd.read_sql(query, con)
data

,d1,d2
0,2023-01-04 11:45:36,2023-01-04 11:45:36


In [ ]:
query = "SELECT	d1,	typeof(d1),	d2,	typeof(d2) FROM datetime_text"
data = pd.read_sql(query, con)
data

,d1,typeof(d1),d2,typeof(d2)
0,2023-01-04 11:45:36,text,2023-01-04 11:45:36,text


In [ ]:
cur.execute("CREATE TABLE datetime_int (d1 int)")
cur.execute("""INSERT INTO datetime_int (d1)
               VALUES	(strftime('%s','now'))
               """)
con.commit()

In [ ]:
query = "SELECT d1 FROM datetime_int"
data = pd.read_sql(query, con)
data

,d1
0,1672832736


In [ ]:
query = "SELECT datetime(d1,'unixepoch') FROM datetime_int"
data = pd.read_sql(query, con)
data

,"datetime(d1,'unixepoch')"
0,2023-01-04 11:45:36


## CREATE TABLE

Each column has a name, data type, and the column constraint. SQLite supports PRIMARY KEY, UNIQUE, NOT NULL, and CHECK column constraints.

In [ ]:
cur.execute("""CREATE TABLE contacts (contact_id INTEGER PRIMARY KEY,
                                      first_name TEXT NOT NULL,
                                      last_name TEXT NOT NULL,
                                      email TEXT NOT NULL UNIQUE,
                                      phone TEXT NOT NULL UNIQUE)
                                      """)
cur.execute("""CREATE TABLE groups (group_id INTEGER PRIMARY KEY,
                                    name TEXT NOT NULL)
                                    """)
con.commit()

## PRIMARY KEY

In [ ]:
cur.execute("""CREATE TABLE countries (country_id INTEGER PRIMARY KEY,
                                       name TEXT NOT NULL)
                                       """)
con.commit()

Another way to set primary key:

In [ ]:
cur.execute("""CREATE TABLE languages (language_id INTEGER,
                                       name TEXT NOT NULL,
                                       PRIMARY KEY (language_id))
                                       """)
con.commit()

However, for tables that the primary keys consist of more than one column, you must use PRIMARY KEY table constraint to define primary keys.

The following statement creates the country_languages table whose primary key consists of two columns.

In [ ]:
cur.execute("""CREATE TABLE country_languages (country_id INTEGER NOT NULL,
                                               language_id INTEGER NOT NULL,
                                               PRIMARY KEY (country_id, language_id),
                                               FOREIGN KEY (country_id) REFERENCES countries (country_id)
                                               ON DELETE CASCADE ON UPDATE NO ACTION,
                                               FOREIGN KEY (language_id) REFERENCES languages (language_id)
                                               ON DELETE CASCADE ON UPDATE NO ACTION)
                                               """)
con.commit()

In [ ]:
cur.execute("""CREATE TABLE supplier_groups (group_id integer PRIMARY KEY,
                                             group_name text NOT NULL)
                                             """)

cur.execute("""CREATE TABLE suppliers (supplier_id   INTEGER PRIMARY KEY,
                                       supplier_name TEXT    NOT NULL,
                                       group_id      INTEGER NOT NULL,
                                       FOREIGN KEY (group_id) REFERENCES supplier_groups (group_id))
                                       """)

con.commit()

The supplier_groups table is called a parent table, which is the table that a foreign key references. The suppliers table is known as a child table, which is the table to which the foreign key constraint applies.

The group_id column in the supplier_groups table is called the parent key, which is a column or a set of columns in the parent table that the foreign key constraint references. Typically, the parent key is the primary key of the parent table.

The group_id column in the suppliers table is called the child key. Generally, the child key references to the primary key of the parent table.

In [ ]:
cur.execute("""INSERT INTO supplier_groups (group_name)
               VALUES ('Domestic'), ('Global'), ('One-Time')
               """)

con.commit()

In [ ]:
query = "SELECT	* FROM supplier_groups"
data = pd.read_sql(query, con)
data

,group_id,group_name
0,1,Domestic
1,2,Global
2,3,One-Time


In [ ]:
cur.execute("""INSERT INTO suppliers (supplier_name, group_id)
               VALUES ('HP', 2)
               """)

con.commit()

In [ ]:
query = "SELECT	* FROM suppliers"
data = pd.read_sql(query, con)
data

,supplier_id,supplier_name,group_id
0,1,HP,2


In [ ]:
cur.execute("""INSERT INTO suppliers (supplier_name, group_id)
               VALUES ('ABC Inc.', 4)
               """)

con.commit()

IntegrityError: ignored

SQLite checked the foreign key constraint, rejected the change, and issued the error above.

To specify how foreign key constraint behaves whenever the parent key is deleted or updated, you use the ON DELETE or ON UPDATE actionç

In [ ]:
cur.execute("DROP TABLE suppliers")

cur.execute("""CREATE TABLE suppliers (supplier_id   INTEGER PRIMARY KEY,
                                       supplier_name TEXT    NOT NULL,
                                       group_id      INTEGER,
                                       FOREIGN KEY (group_id)
                                       REFERENCES supplier_groups (group_id)
                                       ON UPDATE SET NULL
                                       ON DELETE SET NULL)
                                       """)

con.commit()

In [ ]:
cur.execute("""INSERT INTO suppliers (supplier_name, group_id)
               VALUES('XYZ Corp', 3)
               """)

cur.execute("""INSERT INTO suppliers (supplier_name, group_id)
               VALUES('ABC Corp', 3)
               """)

cur.execute("""DELETE FROM supplier_groups
               WHERE group_id = 3""")

con.commit()

In [ ]:
query = "SELECT	* FROM suppliers"
data = pd.read_sql(query, con)
data

,supplier_id,supplier_name,group_id
0,1,XYZ Corp,None
1,2,ABC Corp,None


The SET DEFAULT action sets the value of the foreign key to the default value specified in the column definition when you create the table.

Because the values in the column group_id defaults to NULL, if you delete a row from the supplier_groups table, the values of the group_id will set to NULL.

In [ ]:
cur.execute("DROP TABLE suppliers")

cur.execute("""CREATE TABLE suppliers (supplier_id   INTEGER PRIMARY KEY,
                                       supplier_name TEXT    NOT NULL,
                                       group_id      INTEGER,
                                       FOREIGN KEY (group_id)
                                       REFERENCES supplier_groups (group_id)
                                       ON UPDATE RESTRICT
                                       ON DELETE RESTRICT)
                                       """)

cur.execute("""INSERT INTO suppliers (supplier_name, group_id)
               VALUES('XYZ Corp', 1)""")

con.commit()

In [ ]:
cur.execute("""DELETE FROM supplier_groups
               WHERE group_id = 1""")

con.commit()

IntegrityError: ignored

In [ ]:
cur.execute("""DELETE FROM suppliers
               WHERE group_id = 1""")

con.commit()

In [ ]:
cur.execute("""DELETE FROM supplier_groups
               WHERE group_id = 1""")

con.commit()

The CASCADE action propagates the changes from the parent table to the child table when you update or delete the parent key.

In [ ]:
cur.execute("DELETE FROM supplier_groups")

cur.execute("""INSERT INTO supplier_groups (group_name)
               VALUES ('Domestic'), ('Global'), ('One-Time')
               """)

con.commit()

In [ ]:
query = "SELECT	* FROM supplier_groups"
data = pd.read_sql(query, con)
data

,group_id,group_name
0,1,Domestic
1,2,Global
2,3,One-Time


In [ ]:
cur.execute("DROP TABLE suppliers")

cur.execute("""CREATE TABLE suppliers (supplier_id INTEGER PRIMARY KEY,
                                       supplier_name TEXT NOT NULL,
                                       group_id INTEGER,
                                       FOREIGN KEY (group_id)
                                       REFERENCES supplier_groups (group_id)
                                       ON UPDATE CASCADE
                                       ON DELETE CASCADE)
                                       """)

con.commit()

In [ ]:
cur.execute("""INSERT INTO suppliers (supplier_name, group_id)
               VALUES('XYZ Corp', 1)""")

cur.execute("""INSERT INTO suppliers (supplier_name, group_id)
               VALUES('ABC Corp', 2)""")

con.commit()

In [ ]:
query = "SELECT	* FROM suppliers"
data = pd.read_sql(query, con)
data

,supplier_id,supplier_name,group_id
0,1,XYZ Corp,1
1,2,ABC Corp,2


In [ ]:
cur.execute("""UPDATE supplier_groups
               SET group_id = 100
               WHERE group_name = 'Domestic'""")

con.commit()

In [ ]:
query = "SELECT	* FROM suppliers"
data = pd.read_sql(query, con)
data

,supplier_id,supplier_name,group_id
0,1,XYZ Corp,100
1,2,ABC Corp,2


## UNIQUE

A UNIQUE constraint ensures all values in a column or a group of columns are distinct from one another or unique.

CREATE TABLE table_name (column_name type UNIQUE)

CREATE TABLE contacts(contact_id INTEGER PRIMARY KEY, first_name TEXT, last_name TEXT, email TEXT NOT NULL UNIQUE)

In [ ]:
# CREATE TABLE contacts(contact_id INTEGER PRIMARY KEY,
#                       first_name TEXT,
#                       last_name TEXT,
#                       email TEXT NOT NULL UNIQUE)

# To multiple constraints:

# CREATE TABLE shapes(shape_id INTEGER PRIMARY KEY,
#                     background_color TEXT,
#                     foreground_color TEXT,
#                     UNIQUE(background_color,foreground_color) )

SQLite treats all NULL values are different, therefore, a column with a UNIQUE constraint can have multiple NULL values.